In [0]:
import yaml
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import col

# 1. Konfiguracja Środowiska
try:
    ENV = dbutils.widgets.get("env_name")
except Exception:
    ENV = 'TEST'
    
# 2. Wczytanie Konfiguracji
try:
    with open('../../config/config.yaml', 'r') as file:
        full_config = yaml.safe_load(file)
except FileNotFoundError:
    print("BŁĄD: Plik 'config.yaml' nie został znaleziony! Sprawdź ścieżkę.")
    raise

CFG = full_config.get(ENV)
if not CFG:
    raise ValueError(f"Nie znaleziono konfiguracji dla środowiska: {ENV} w pliku YAML.")

catalog_name = CFG['catalog_name']
schema_name = CFG['schema_name']
volume_name = CFG['volume_name']

# UŻYJEMY POPRAWIONEJ ŚCIEŻKI ZGODNEJ ZE ZDJĘCIEM I BŁĘDEM
# Folder docelowy to 'yfinance_bronze_data', który jest tabelą Delta
base_output_directory = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/yfinance_bronze_data"
print(f"Ścieżka do tabeli Delta Bronze: {base_output_directory}")

In [0]:
try:
    # Używamy formatu "delta" do odczytu folderu Delta Lake
    df_bronze = (
        spark.read.format("delta") 
        .load(base_output_directory) # Ładowanie ze ścieżki do folderu Delta
        .withColumn("Date", col("Date").cast("date"))
    )
    
    print(f"Pomyślnie wczytano dane. Liczba wierszy: {df_bronze.count()}")
    df_bronze.printSchema()
    
    df_bronze.select("Ticket", "Date", "Close").show(5)

except Exception as e:
    print(f"BŁĄD: Wystąpił problem podczas ładowania danych: {e}")